# Objective: To use gguf model locally with langchain 
OllamaLLm and HuggingFaceEmbedding: mistralai/Mistral-7B-Instruct-v04

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [1]:
from langchain_community.document_loaders import PyPDFLoader
file_path = (r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\3_Text_query_bot\_docs\Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

### Cutting down time whilst createing embeddings

In [6]:
## Not local
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Create retriever

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [8]:
# relevant_docs = retriever.get_relevant_documents("Definition")
# print(relevant_docs[0].page_content)

c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Leave Policy  
 Version  HRLP24  
Issue Date  1-Apr-2024 
Effective Date  1-Jan-2024  
This version supersedes all previous versions with a lesser version number  
 
Page 1 of 4 
 
1.0 OBJECTIVE : Leave is a period of rest and relaxation  provided at intervals during  
work periods. For convenience in terms of reference, it is divided into categories  
such as Privilege Leave (PL), Casual Leave (CL), Sick Leave (SL) and Special Leave  
    
2.0 DEFINITIONS : 
2.1 Leave Year:  The Leave Year is the calendar year from 1 Jan uary  to 31 Dec ember . 
All calculations of earning, credit, availment, balance etc. of leave will be done 
with respect to each Leave Year  
2.2 Entitlement: This refers to the amount , or the numb er of days  of Leave (of 
different types ) that an employee gets in a Leave Year as part of conditions of 
service , subject to certain conditions such as full attendance on full salary. For 
example,  the entitlement of PL is 21 days in a Leave Year


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### LLM Used:

## 1)Model: mistral_7B_v0.3
    Langchain + CTransformer
    Size: 4.07 GB 
    RAM usage: how to find??
    Response time:
    
- quantized model with langchain
- on cpu without nvidia gpu
- cpu -- 16 gb RAM
- download the gguf model
- What is quantization --> the higger bit quant --> more ram

In [10]:
from langchain_community.llms import CTransformers
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

model_dir =  r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\mistral_7B_v0.3"
model_file = "Mistral-7B-Instruct-v0.3.Q4_K_M.gguf"

config = {'context_length': 2048}


llm = CTransformers(model= model_dir, model_file = model_file, callbacks=[StreamingStdOutCallbackHandler()], config= config)

In [11]:
# llm.invoke("Make a poem on birds")



In the tranquil skies so bright,
A symphony of feathers take flight.
Their melodious songs fill the air,
A gentle whisper from afar.

From the smallest hummingbird's flit,
To the majestic eagle's might,
Each bird in its own way, so divine,
An ode to life and the great design.

The robin, a harbinger of spring,
Its red breast adorning its wings,
Sings with joy as it takes its place,
In nature's grand and wondrous space.

The sparrow, humble yet steadfast,
Lives in harmony, never boastful or hasty,
A lesson for us all to learn,
To cherish what is precious, but still unseen.

The swift and graceful swallows fly,
With a speed that leaves the eye to wonder why,
Like shadows darting across the sunlit field,
Their agility and beauty never fail to yield.

The soaring eagle on its wings so wide,
Majestic and powerful, it takes pride,
Its

"\n\nIn the tranquil skies so bright,\nA symphony of feathers take flight.\nTheir melodious songs fill the air,\nA gentle whisper from afar.\n\nFrom the smallest hummingbird's flit,\nTo the majestic eagle's might,\nEach bird in its own way, so divine,\nAn ode to life and the great design.\n\nThe robin, a harbinger of spring,\nIts red breast adorning its wings,\nSings with joy as it takes its place,\nIn nature's grand and wondrous space.\n\nThe sparrow, humble yet steadfast,\nLives in harmony, never boastful or hasty,\nA lesson for us all to learn,\nTo cherish what is precious, but still unseen.\n\nThe swift and graceful swallows fly,\nWith a speed that leaves the eye to wonder why,\nLike shadows darting across the sunlit field,\nTheir agility and beauty never fail to yield.\n\nThe soaring eagle on its wings so wide,\nMajestic and powerful, it takes pride,\nIts"

### Creating Prompt

In [12]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [13]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate

## Exploring Retrievers

In [14]:
# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [16]:
retriever = vectorstore.as_retriever()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Chain

In [20]:
chain = (
    {"context": retriever| format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result=chain.invoke("Tell me more about PL in 10 words only")


Answer: Earned leave that must be taken every 2 years, credited annually.

In [18]:
result


"\nPrivilege Leave (PL) is a type of earned leave provided by an organization to its employees. The entitlement and earning of PL are based on the number of months of service rendered by the employee. For example, an employee earns PL at the rate of 1.75 days for every month of service.\n\nThe credit of PL is given to the employee's Leave account at the end of each Leave Year, which starts from the 1st of January of the next Leave Year. Employees are required to compulsorily avail at least 15 days of PL in a block of 2 years. This period is referred to as the first block of 2 years and starts from the January when 21 days of leave are earned for those who have recently joined the organization. Any unavailed compulsory PL will lapse in the Leave Year following the year in which compulsory PL was due.\n\nEmployees can avail advance PL through an appropriate channel, but the settled amount will be recovered from their PL balance that will be credited in the next Leave Year. The maximum nu

In [19]:
import re
answer =re.sub(r'[0-9]','',questions)
question_list = answer.split('\n')
question_list
chain.invoke(question_list[0])

NameError: name 'questions' is not defined

In [ ]:
''' Taking too much time'''
# answer_list = []
# for question in question_list:
#     ans = chain.invoke(question)
#     answer_list.append(ans)
print("ANSWERS-----")
# print(answer_list)
--------------------------------------------(2)--------------------------------------------